In [1]:
import pandas as pd
import numpy as np
import scanpy as sc
import requests
import json
import os, sys
#sys.path.append('/home/fvalle/phd/master_thesis/hsbm/')

/opt/conda/lib/python3.7/site-packages/anndata/_core/anndata.py:21: FutureWarning: pandas.core.index is deprecated and will be removed in a future version.  The public classes are available in the top-level namespace.
  from pandas.core.index import RangeIndex


# Download data

In [2]:
#path/to/files/downloaded/from 
#https://doi.org/10.6084/m9.figshare.5330593
#https://figshare.com/articles/Data_record_2/5330575
working_dir = "/home/jovyan/work/phd/datasets/merged/"
os.chdir(working_dir)

In [5]:
pd.read_csv("mainTable.csv")

,Unnamed: 0,TCGA-B6-A0RV-01A-11R-A084-07,TCGA-BH-A1EU-01A-11R-A137-07,TCGA-E9-A1RB-11A-33R-A157-07,TCGA-B6-A0RS-01A-11R-A084-07,TCGA-A8-A06N-01A-11R-A00Z-07,TCGA-AR-A2LL-01A-11R-A180-07,TCGA-BH-A0HL-01A-11R-A10U-07,TCGA-A8-A09Q-01A-11R-A00Z-07,TCGA-BH-A1FR-01A-11R-A13Q-07,...,GTEX-11WQK-2926-SM-5EQKT,GTEX-13111-2326-SM-5LZUQ,GTEX-13QJC-2326-SM-5Q5CD,GTEX-ZA64-1726-SM-5CVMF,GTEX-13O21-1926-SM-5MR39,GTEX-11NV4-1926-SM-5N9D4,GTEX-12WSL-2126-SM-5GCNG,GTEX-XMK1-2626-SM-4B65R,GTEX-WOFM-2126-SM-4OOSR,GTEX-13VXT-1726-SM-5IFF5
0,A2ML1,1.71,4.66,6.52,0.00,0.65,1.95,0.36,1.17,3.59,...,703.28,323.03,819.30,6792.79,1037.29,115.16,529.06,2240.11,15499.21,536.45
1,A4GNT,0.34,0.74,2.10,0.00,0.00,0.65,1.23,0.77,0.33,...,0.04,1.22,0.87,1.30,1.03,0.00,0.00,0.13,0.68,1.22
2,AADAC,0.30,59.97,223.41,17.38,0.00,2.16,0.00,0.71,0.00,...,0.00,121.79,24.99,18.70,71.50,291.04,26.28,13.52,37.32,25.91
3,AARD,140.04,961.07,167.90,1001.93,138.10,69.52,198.47,229.72,67.59,...,0.61,14.24,12.74,7.51,5.15,8.78,8.71,8.19,4.54,4.90
4,ABAT,723.08,753.83,214.27,23.08,301.33,579.04,444.72,84.04,144.01,...,72.01,71.50,156.59,45.85,150.17,162.14,43.02,36.79,109.66,269.60
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2995,ZNF560,0.00,0.71,11.82,0.00,0.00,0.00,1.45,0.37,0.32,...,0.00,0.00,0.66,0.62,0.00,0.00,0.00,0.26,0.99,0.00
2996,ZNF668,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,7.94,0.00,10.24,0.00,0.00,0.00,0.00,4.78
2997,ZNF750,92.70,63.45,56.28,234.57,142.01,20.11,33.54,15.45,21.32,...,144.01,718.08,1674.06,529.06,1674.06,2468.49,1540.37,529.06,1674.06,2855.44
2998,ZNF812,4.74,19.53,52.82,12.74,2.51,1.83,1.16,16.27,7.06,...,156.59,126.12,199.85,429.54,669.92,8.06,1498.22,224.97,146.03,256.78


In [ ]:
files = filter(lambda f: "fpkm" in f, os.listdir("data"))

In [ ]:
df_gtex=pd.read_csv("https://storage.googleapis.com/gtex_analysis_v8/annotations/GTEx_Analysis_v8_Annotations_SampleAttributesDS.txt", sep='\t', index_col=0)

In [ ]:
filters = {
    "op": "and",
    "content":[
        {
        "op": "in",
        "content":{
            "field": "cases.project.program.name",
            "value": ["TCGA"]
            }
        }
        
    ]
}
params = {
    "filters": json.dumps(filters),
    "fields": "primary_site,disease_type,submitter_id",
    "format": "TSV",
    "size": "10000000"
    }
response = requests.get("https://api.gdc.cancer.gov/cases", headers = {"Content-Type": "application/json"}, params = params)
with open("files.txt","w") as file:
    file.write(response.content.decode("utf-8"))
    file.close()
df_tcga = pd.read_csv("files.txt", sep='\t').set_index("submitter_id")

In [ ]:
df=pd.DataFrame()

In [ ]:
for file in files:
    df = df.append(pd.read_csv("data/%s"%file, sep='\t', index_col=0).drop('Entrez_Gene_Id',1).transpose(), sort=True)
df = df.transpose()
df = df.dropna(how='any', axis=0) # drop genes not always determined

In [ ]:
df_files = pd.DataFrame(index=df.columns)

In [ ]:
def get_site(file):
    if 'GTEX' in file:
        return df_gtex.at[file, 'SMTS']
    if 'TCGA' in file:
        return df_tcga.at[file[:12],'primary_site']

def get_source(file):
    if 'GTEX' in file:
        return 'gtex'
    if 'TCGA' in file:
        return 'tcga'

In [ ]:
df_files.insert(0, 'primary_site', [get_site(file) for file in df.columns])
df_files.insert(1, 'dataset', [get_source(file) for file in df.columns])

In [ ]:
#df.to_csv("mainTable.csv", index=True, header=True)
df_files.to_csv("files.dat", index=True, header=True)

## Split / shuffle and select

In [ ]:
#df = pd.read_csv("mainTable.csv", index_col=0)
df_files = pd.read_csv("files.dat", index_col=0)
df.head()

In [ ]:
df_files.replace('Uterus, NOS', 'Uterus', inplace=True)
df_files.replace('Bronchus and lung', 'Lung', inplace=True)
df_files.replace('Liver and intrahepatic bile ducts', 'Liver', inplace=True)
df_files.replace('Prostate gland', 'Prostate', inplace=True)
df_files.replace('Thyroid gland', 'Thyroid', inplace=True)
df_files.replace('Base of Tongue', 'Salivary Gland', inplace=True)
df_files.replace('Bones, joints and articular cartilage of other and unspecified sites', 'Salivary Gland', inplace=True)
df_files.replace('Floor of mouth', 'Salivary Gland', inplace=True)
df_files.replace('Gum', 'Salivary Gland', inplace=True)
df_files.replace('Hypopharynx', 'Salivary Gland', inplace=True)
df_files.replace('Larynx', 'Salivary Gland', inplace=True)
df_files.replace('Lip', 'Salivary Gland', inplace=True)
df_files.replace('Oropharynx', 'Salivary Gland', inplace=True)
df_files.replace('Other and ill-defined sites in lip, oral cavity and pharynx', 'Salivary Gland', inplace=True)
df_files.replace('Other and unspecified parts of mouth', 'Salivary Gland', inplace=True)
df_files["tissue_hd"]=df_files["primary_site"]+"_"+df_files["dataset"]
df_files['primary_site'].unique()

In [ ]:
#https://www.ncbi.nlm.nih.gov/pmc/articles/PMC5903355/
sites_with_enough_statistics = ['Breast', 'Colon', 'Liver', 'Esophagus', 'Prostate', 'Stomach', 'Lung', 'Uterus']

In [ ]:
rs = np.random.RandomState(seed=42)
samples = pd.DataFrame(columns=df_files.columns)
for site in sites_with_enough_statistics:
    samples = samples.append(df_files[(df_files['primary_site']==site) & (df_files['dataset']=='tcga')].sample(50, random_state=rs))
    samples = samples.append(df_files[(df_files['primary_site']==site) & (df_files['dataset']=='gtex')].sample(50, random_state=rs))

In [ ]:
samples.groupby('primary_site').count()

In [ ]:
samples.groupby('dataset').count()

In [ ]:
df[df.columns[~df.columns.isin(samples.index) & df.columns.isin(df_files[df_files["primary_site"].isin(sites_with_enough_statistics)].index)]].to_csv("mainTable_test.csv", index=True, header=True)

In [ ]:
df[df.columns[df.columns.isin(samples.index)]].to_csv("mainTable_train.csv", index=True, header=True)

In [ ]:
df_files["tissue_hd"].unique()

In [ ]:
df_files.to_csv("files.dat", index=True, header=True)

# Use scanpy to filter HVG

In [ ]:
adata = sc.AnnData(df[samples.index].transpose(), obs=samples)
adata_log = sc.pp.log1p(adata, copy=True)

In [ ]:
sc.pp.highly_variable_genes(adata_log, n_top_genes=3000, n_bins=50)

In [ ]:
sc.pl.highly_variable_genes(adata_log, log=True, save='hvg.pdf')

In [ ]:
hvg = adata_log.var[adata_log.var['highly_variable']==True].index
samples = adata_log.obs.index

# Use SBM

In [ ]:
from sbmtm import sbmtm

In [ ]:
model = sbmtm()

In [ ]:
model.make_graph_from_BoW_df(df.loc[hvg, samples])

In [ ]:
model.save_graph("graph.xml.gz")